In [3]:
import requests
import os
import bs4
import re
import datetime
import csv

I am setting up this script to only pull filings posted since last run 

In [4]:
last_time_run = datetime.datetime.strptime('01/01/2018', "%m/%d/%Y").date()

In [5]:
form = 'DEF 14A'
base_url = "https://www.sec.gov/cgi-bin/srch-edgar"

In [6]:
url = "%s?text=%s&first=2018&last=2018" %(base_url, form)
list_of_forms = requests.get(url)
soup = bs4.BeautifulSoup(list_of_forms.text, 'html.parser')

In [22]:
table_of_filings = (soup.findAll("table")[4]).findAll("tr")
num_filings = len(table_of_filings)

Loop over all filings on first page

In [93]:
for i in range(1, num_filings):
    row = table_of_filings[i]
    cols = row.findAll("td")
    # Get Company Name and Filing Date
    comp_name = cols[1].get_text()
    filing_date = datetime.datetime.strptime(cols[4].get_text(), "%m/%d/%Y").date()
    # Only grab those filings that have been posted since we last pulled
    if filing_date >= last_time_run:
        txt_url = cols[2].find()['href']
        comp_def14a_html = requests.get('https://www.sec.gov/%s' %txt_url).text
        #form_soup = bs4.BeautifulSoup(comp_def14a_html, 'html.parser')
        #comp_def14a = form_soup.get_text()
        comp_def14a = re.sub('<[^<]+?>', '', comp_def14a_html)
        # If filing doesn't contain median we are not going to find the salary
        if comp_def14a.find("median") == -1:
            pass
            # Write to out file
            #with open('comps_no_median_salary.csv','a', newline='') as nosal_file:
            #    no_sal_writer = csv.writer(nosal_file, delimiter=',')
            #    no_sal_writer.writerow([comp_name, filing_date.year])
            # Typically reported in section called:
                # CEO Pay Ratio or Pay Ratio Disclosure
        else:
            pay_ratio = comp_def14a.rfind("Pay Ratio")
            pay_ratio_caps = comp_def14a.rfind("PAY RATIO")
            # Fairly confident these won't have info, but come back to it
            if pay_ratio == -1 and pay_ratio_caps == -1:
                pass
            # Need to get the section with pay ratio and find median salary
            # May need to build classifier but pretty sure I can retrieve it
            else:
                print(comp_name)
                ratio_sec = max(pay_ratio, pay_ratio_caps)
                pay_section = comp_def14a[ratio_sec : ratio_sec + 5000]
                pay_section = pay_section.strip("\n")
                symbs = ["&", "#", "&nbsp;", "nbsp", "\n", ';']
                for s in symbs:
                    pay_section = pay_section.replace(s, "")
                    
                all_salaries = re.findall('\$\d{1,3},\d{3,3},?\d*', pay_section) 
                print('all salaries', all_salaries)
                ratio_colon = re.findall('\d{2,4}:\d{1}', pay_section)
                ratio_to = re.findall('\d+\.?\d*\sto\s\d', pay_section)   
                if ratio_colon != [] or ratio_to != []:
                    if ratio_colon != []:
                        ratio_str = ratio_colon[0]
                        ceo_mult = float(ratio_str[0:ratio_str.find(':')])
                    elif ratio_to != []:
                        ratio_str = ratio_to[0]
                        ceo_mult = float(ratio_str[0:ratio_str.find(' to ')])
                    
                    print('ratio', ceo_mult)
                    emp_salary = -1
                    ceo_salary = -1
                    num_salaries = [float(num.replace('$', '').replace(',', '')) for num in all_salaries]
                    num_salaries.sort()
                    for i in range(len(num_salaries) - 1):
                        for j in range(i + 1, len(num_salaries)):
                            lower_sal = num_salaries[i]
                            higher_sal = num_salaries[j]
                            if abs(higher_sal / lower_sal - ceo_mult) < 1:
                                ceo_salary = higher_sal
                                emp_salary = lower_sal
                    print('emp_salary', emp_salary)
                    print('ceo_salary', ceo_salary)
                    if emp_salary == -1 and ceo_salary == -1:
                        print(pay_section)
                    
                else:
                    print('Cant use ratio') 
                    print(pay_section)

                #six_figures = re.findall('\$\d{1,3},\d{3,3}', pay_section)
                #nine_figures = re.findall('\$\d{1,3},\d{3,3},\d{3,3}?', pay_section)
                #for sal in six_figures:
                #    for nine_sal in nine_figures:
                #        if sal in nine_sal:
                #            six_figures.remove(sal)
                #            break                
                #if len(six_figures) == 1:
                #    med_emp_salary = six_figures[0]
                #else:
                #    print('pass')
                    #emp_ind = (pay_section.find('annual total compensation') + pay_section.find('employees')) / 2
                    #print('Employee Sentence position', emp_ind)
                    #dist_to_ind = 10000
                    #winning_index = -1
                    #for i, sal in enumerate(six_figures):
                    #    dist = abs(pay_section.find(sal)- emp_ind)
                    #    if dist < dist_to_ind:
                    #        dist_to_ind = dist
                    #        winning_index = i
                    #med_emp_salary = six_figures[winning_index]
                #print('Median Employee Salary', med_emp_salary)
                
                
                if len(nine_figures) == 1:
                    ceo_salary = nine_figures[0]
                else:
                    pass

1ST CONSTITUTION BANCORP
all salaries ['$53,278', '$1,487,675', '$1,000,000']
ratio 28.0
emp_salary 53278.0
ceo_salary 1487675.0
1ST SOURCE CORP
all salaries ['$1,855,632', '$45,286', '$62,500160', '$62,500160']
ratio 41.0
emp_salary 45286.0
ceo_salary 1855632.0
22nd Century Group, Inc.
all salaries ['$711,375', '$36,608', '$345,987', '$1,903,500', '$128,836', '$128,836', '$64,418']
Cant use ratio
Pay RatioWe have determined the internal pay ratio betweenour chief executive officerrsquos total compensation and the median annual total compensation of all employees (except for thechief executive officer). We identified the ldquoMedian Employeerdquo based on the total annualized compensation of all full-timeand part-time employees employed by us on December 1, 2017. Our chief executive officer had annual compensation of $711,375 andour Median Employee had annual compensation of $36,608. Therefore, our chief executive officerrsquos annual total compensationis approximately nineteen (19) ti

ABBOTT LABORATORIES
all salaries ['$12,597', '$75,679,', '$26,863,000', '$14,545,000', '$33,219,000', '$18,191,000']
ratio 251.0
emp_salary -1
ceo_salary -1
PAY RATIO   In2017, we compared CEO pay to that of our median employee. To identify our median employee, we first excluded all 3,653 employees who are employed in Bolivia (213), Egypt (332),Indonesia (633), Mexico (1,122), Peru (1,329), and Venezuela (24), representing less than 5% of our global workforce of 89,647 employees as of October1, 20171. We then examinedthe 2017 base salary of all remaining employees globally, excluding our CEO, who were employed by us on October1, 2017. We annualized the base salary of all permanent employees who were hiredin 2017 but did not work for the entire year. The base salary for employees outside of the U.S. was converted to U.S. dollars. Afteridentifying the median employee, we collected annual total compensation for this employee using the same methodology we use for our named executive office

ABIOMED INC
all salaries ['$122,674', '$10,304,726', '$122,674', '$17,724,']
ratio 84.0
emp_salary 122674.0
ceo_salary 10304726.0
ABRAXAS PETROLEUM CORP
all salaries ['$452,333', '$91,834', '$1,099,937', '$94,454', '$8,846', '$304,308,', '$2,620', '$9,450', '$3,000', '$1,454,166', '$452,333']
Cant use ratio
Pay RatioPresidentand CEOMedianEmployee Base Salary$452,333$91,834 Non-Equity Incentive Plan Compensation322,000151 Discretionary Bonus313,154(1)2,620(2) All Other Compensation12,450(3)151    $1,099,937$94,454     (1)This amount reflects a discretionary holiday bonus and a one-time discretionary bonus of $8,846 and $304,308, respectively,awarded to Mr.Watson in 2017. (2)This amount reflects a $2,620 safety bonus to the median employee. (3)This amount represents a $9,450 contribution by Abraxas to Mr.Watson146s 401(k) plan and a $3,000 contribution to Mr.Watson146s health savingsaccounts for 2017.   17 Table of Contents Elements of Executive Compensation Executive compensation consis

Acadia Healthcare Company, Inc.
all salaries ['$25,830', '$8,633,100', '$124,000', '$159,985', '$283,985']
ratio 334.0
emp_salary 25830.0
ceo_salary 8633100.0
ACADIA PHARMACEUTICALS INC
all salaries ['$15,158,146', '$283,058,', '$50,000', '$30,000', '$20,000', '$10,000', '$20,000', '$10,000', '$10,000', '$5,000']
ratio 54.0
emp_salary 283058.0
ceo_salary 15158146.0
ACADIA REALTY TRUST
all salaries ['$4,514,163', '$104,500', '$145,000,', '$65,000', '$80,000', '$12,500,', '$20,000', '$100,000']
Cant use ratio
Pay RatioFor 2017, the Company estimated the ratio between the chief executive officer8217s total compensation and the median annual total compensation of all employees (except the chief executive officer). In searching for the median employee, base compensation totals in 2017 were considered. The 8220Median Employee8221 was identified using the base compensation of all full-time, part-time, and temporary employees employed by the Company on December 31, 2017, then the Median Employ

ACCESS NATIONAL CORP
all salaries ['$68,234', '$692,471', '$36,000', '$12,000', '$48,000', '$192,000']
Cant use ratio
Pay RatioAs required by SEC rules,we are providing our shareholders the following information about the relationship of the annual total compensation of our employeesand the annual total compensation of our CEO.We determined that the2017 annual total compensation of our median compensated employee, from of all our employees who were employed as of December 31,2017, other than our CEO, was $68,234 our CEOrsquos 2017 annual total compensation was $692,471 and the ratio of these amountswas 1:10.As of December 31, 2017,our total population consisted of 397 employees, all located in the United States. This population consisted of all of ourfull-time, part-time and temporary employees.To identify the mediancompensated employee, we used the total actual wages for the period from January 1, 2017 to December 31, 2017 as reported to theInternal Revenue Service on Box 1 of Form W-

ACELRX PHARMACEUTICALS INC
all salaries []
Cant use ratio
Pay Ratio measure in making compensation decisions.29Grants of Plan Based Awards in Fiscal 2017 The following table provides information with regard to each grant of plan-based award made to a named executive officer under any plan during the fiscal year ended December31, 2017.											Grant Date 																																													Number of			Securities			Underlying			Options()(2) 									Exercise			or Base			Price of			Option			Awards			($/Sh)(3) 									Grant Date			Fair Value			of Option			Awards($) 																															Estimated Future Payouts Under			Non-Equity Incentive Plan			Awards(1) 																																																	Name																		Threshold($)									Target($) 									Maximum($) 																																																	Vincent J. Angotti																																																																																																	Stock Options												3/6/201

Addus HomeCare Corp
all salaries ['$45,000', '$1,500', '$20,000', '$15,000,', '$10,000,', '$7,500', '$10,000,', '$1,500', '$1,000', '$74,989,']
ratio 188.0
emp_salary -1
ceo_salary -1
Pay Ratio was 188:1. Each individual146s total annual compensation can be comprised of differentcompensation elements and is dependent on where the individual works.  Please keep in mind that under the SEC146s rules andguidance, there are numerous ways to determine the compensation of a company146s median employee, including the employee population sampled, the elements of   38  pay and benefits used, any assumptions made and the use of statistical sampling. In addition, no two companies have identical employee populations or compensation programs, and pay, benefits andretirement plans differ by country even within the same company. As such, our pay ratio may not be comparable to the pay ratio reported by other companies.  2017 Director Compensation  Consistent with the Company146s independentdirector com

Advanced Disposal Services, Inc.
all salaries ['$55,102,', '$3,829,281']
ratio 69.0
emp_salary 55102.0
ceo_salary 3829281.0
ADVANCED DRAINAGE SYSTEMS, INC.
all salaries ['$4,062,900', '$52,385', '$100,000']
ratio 78.0
emp_salary 52385.0
ceo_salary 4062900.0
Advanced Emissions Solutions, Inc.
all salaries ['$117,584', '$1,656,921']
Cant use ratio
Pay RatioAs required by Section 953(b) of the Dodd-Frank Wall Street Reform and Consumer Protection Act of 2010 and Item 402(u) of Regulation S-K, we are providing the following information with respect to our last completed fiscal year (2017). The pay ratio reported below is a reasonable estimate calculated in a manner consistent with Item 402(u) of Regulation S-K.We have estimated the median of the 2017 annual total compensation of our employees, excluding our CEO, to be $117,584. The annual total compensation of our CEO was $1,656,921. The ratio of the annual total compensation of our CEO to the estimated median of the annual total compensat

ADVANCED MICRO DEVICES INC
all salaries ['$89,909', '$10,894,821']
ratio 121.0
emp_salary 89909.0
ceo_salary 10894821.0
AdvanSix Inc.
all salaries ['$98,422', '$3,864,058']
ratio 39.0
emp_salary 98422.0
ceo_salary 3864058.0


In [9]:
has_median = comp_def14a.text.find("median")
print(has_median)

AttributeError: 'str' object has no attribute 'text'

In [ ]:
if has_median == -1:
    comps_wo_med.append(comp_name)
else:
    # Look for Pay Ratio section
    

In [70]:
test_that_has = requests.get('https://www.sec.gov/Archives/edgar/data/1551152/000104746918001843/0001047469-18-001843.txt')

In [74]:
section_start = test_that_has.text.find("Required Pay Ratio")

In [75]:
ratio_paragraph = test_that_has.text[section_start:section_start + 1000]

In [76]:
print(ratio_paragraph)

Required Pay Ratio Disclosure    <BR>    </I></B></FONT></P>

<P style="font-family:arial;"><FONT SIZE=2>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;As required by Section&nbsp;953(b) of the Dodd-Frank Wall Street Reform and Consumer Protection Act and Item&nbsp;402(u) of
Regulation&nbsp;S-K, we are providing the following information about the relationship of the annual total compensation of our employees and the annual total compensation of our CEO, Richard
Gonzalez. The pay ratio included in this information is a reasonable estimate calculated in a manner consistent with Regulation&nbsp;S-K Item&nbsp;402(u). The ratio of Mr.&nbsp;Gonzalez's annual
total compensation for 2017, as reported in the Summary Compensation Table in this proxy statement, to the median employee annual total compensation determined on the same basis was 144:1. For 2017,
the annual total compensation of our median employee (other than Mr.&nbsp;Gonzalez) was $157,347. To i

In [91]:
par_lst = ratio_paragraph.split('. ')

In [92]:
print(par_lst)

['Required Pay Ratio Disclosure    <BR>    </I></B></FONT></P>\n\n<P style="font-family:arial;"><FONT SIZE=2>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;As required by Section&nbsp;953(b) of the Dodd-Frank Wall Street Reform and Consumer Protection Act and Item&nbsp;402(u) of\nRegulation&nbsp;S-K, we are providing the following information about the relationship of the annual total compensation of our employees and the annual total compensation of our CEO, Richard\nGonzalez', 'The pay ratio included in this information is a reasonable estimate calculated in a manner consistent with Regulation&nbsp;S-K Item&nbsp;402(u)', "The ratio of Mr.&nbsp;Gonzalez's annual\ntotal compensation for 2017, as reported in the Summary Compensation Table in this proxy statement, to the median employee annual total compensation determined on the same basis was 144:1", 'For 2017,\nthe annual total compensation of our median employee (other than Mr.&nbsp;Gonzalez) was 

In [116]:
for sentence in par_lst:
    if sentence.find("median employee") != -1 and sentence.find("$") != -1:
        #med_salary = re.search('\$[0-9]+', sentence)
        med_salary = re.search('\$\d+,\d+\d+?', sentence)
        print(med_salary.group())

$157,347


In [120]:
comps_w_med.loc[0] = ['AbbVie', med_salary.group()]


In [121]:
comps_w_med.head()

,company,med_emp_salary
0,AbbVie,"$157,347"
